# TVLA using Hamming Weights

In [31]:
import numpy as np
from tqdm import tqdm

from src.data.slices import tvla_slice, random_slice
from src.data.traceloader import AscadRandomKey

In [32]:
ascad = AscadRandomKey()

In [33]:
from src.tvla.t import make_t_test

def prepare_hw(make_test, samples, limit):
    """
    Prepares traces for TVLA using Hamming weights as a key distance measure.
    """
    target = samples.filter_by_hw(True)[:limit]
    rest = samples.filter_by_hw(False)[:limit]

    slc_target = random_slice(target, 2)
    slc_rest = random_slice(rest, 2)

    def means(slc):
        return np.array([(s.mean(axis=0), s.var(axis=0)) for s in slc], dtype=np.float128)

    slc_mv = np.moveaxis(np.array([means(slc_target), means(slc_rest)]), 2, 3)
    test = make_test(slc_mv.shape[2], False)

    return test, slc_mv

In [34]:
def tvla(test, left, right, debug=False):
    """
    Applies some given statistical test against the given samples.

    @param test: statistical test function that takes two samples and returns test values.
    @param left: Four sample sets containing s samples representing a power trace of length t.
    @param right: Two sample sets containing s samples representing a power trace of length t.
    @param p: Decide whether a device fails using (100 * p)%-confidence interval.
    @param debug: Set to True if this function should throw Exceptions upon improperly structured input.
    @return: Whether the device fails on (A against A, A against B).
    """
    if debug:
        assert len(left) == 2
        assert len(right) == 2

    # Four different samples from distribution A.
    a1, a2 = left
    # Two different samples from distribution B. x and y are compared to a and b, respectively.
    _, b2 = right

    # Test A against A, only gather t-values
    aa = test(a1, a2)[0]

    # Test A against B, only gather t-values
    ab = test(a1, b2)[0]

    # Return max t-values for aa and ab
    return np.max(aa), np.max(ab)

### t-value gradient

In [35]:
lim = 10000
default_test, default_slices = prepare_hw(make_t_test, ascad.default.profile, lim)
random_test, random_slices = prepare_hw(make_t_test, ascad.random.profile, lim)

tvla_default = tvla(default_test, default_slices[0], default_slices[1])
tvla_random = tvla(random_test, random_slices[0], random_slices[1])

print("Default", tvla_default)
print("Default", tvla_random)

Default (2.3092872662932, 2.0019751419013834)
Default (1.8979306469207586, 1.817142594502705)


Issue 1: does TVLA compare the entire trace instead of just a sample point distribution?